# Ⅰ.动态网页爬取

In [ ]:
'''
功能：逆向爬取网页--图片
作者：吴小华
时间：2019-7-22 上午
'''
import  requests
import json

#1.读取本地文件为json格式
with open(r'C:\Users\苏哥哥\Desktop\111\pic.txt','r') as pf:
    pic_list = json.load(pf)
print(len(pic_list))

#2.解析出图片url链接
pic_url = [] #定义一个空列表，用来存放图片url
for i in range(len(pic_list)): #提取所有图片url
   pic_url.append(pic_list[i]['cover'])
print(pic_url)

#3.根据url链接爬取图片到本地
for j in range(len(pic_url)):
    pic_html = requests.get(pic_url[j]) #发送图片的url进行请求
    with open(r'C:\Users\苏哥哥\Desktop\111\pic_%d.png'%j,'wb') as pf: #把图片保存到本地，以二进制形式写
        pf.write(pic_html.content)

In [ ]:
'''
功能：BeautifulSoup解析网页
作者：吴小华
时间：2019-7-22 上午
'''
import requests
from bs4 import BeautifulSoup
import chardet

#1.发url请求
url = 'http://www.163.com'
ua = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1'}
resp = requests.get(url,headers = ua,timeout=3)
print(resp.encoding)

#2.转换编码
resp.encoding = chardet.detect(resp.content)['encoding']
print(resp.encoding)

#3.找到主网页所有文本和超链接url
html = resp.content
soup = BeautifulSoup(html,'html.parser') #创建bs对象
soup.prettify #初始化对象
class_data = soup.select('ul.cm_ul_round > li > a') #返回的是tag对象的列表
child_title = []
child_href = []
for i in range(len(class_data)):
    child_title.append(class_data[i].get_text()) #取其中的1个tag进行提取文本信息
    child_href.append(class_data[i].get('href')) #取其中的1个tag进行提取超链接信息
print(child_title)
print(child_href)

#4.爬取子网页url信息
for i in range(10):
    child_page = requests.get(child_href[i])
    child_html = child_page.content
    child_soup = BeautifulSoup(child_html,'html.parser') #创建bs对象
    child_soup.prettify #初始化对象

    child_text = child_soup.select('div.post_text > p')
    child_text_list = []
    for j in range(len(child_text)):
        child_text_list.append(child_text[j].get_text())
    
    with open(r'C:\Users\苏哥哥\Desktop\111\%d.txt'%i,'w') as pf:
        for k in child_text_list:
            pf.write(k)

# Ⅱ.爬取流浪地球

In [ ]:
'''
功能：爬取流浪地球
作者：吴小华
时间：2019-7-22 下午
'''
import requests
from bs4 import BeautifulSoup

#解析url函数
def resp_def(url,ua):
    resp = requests.get(url,headers=ua,timeout = 30)
    return resp

#转换为bs4格式函数
def soup_def(resp):
    #print(resp.encoding)
    html = resp.content
    soup = BeautifulSoup(html,'html.parser')
    soup.prettify()
    return soup

#解析元素函数
def info_def(soup):
    title1 = soup.select('title')
    print(title1[0].get_text())
    #提取名字
    info_name = soup.select('header.main-hd > a.name')
    #提取影评
    info_yp = soup.select('header.main-hd > span')
    #提取短频内容
    info_cont = soup.select('div.short-content')

    #提取url
    info_url = soup.select('header.main-hd > a.avator')
    name_list = []
    title_list = []
    time_list = []
    cont_list = []
    url_list = []
    for i in range(len(info_name)):
        name_list.append(info_name[i].get_text())
        #print(name_list)
        title_list.append(info_yp[i*2].get('title'))
        #time_list.append(info_yp[i*2+1].get_text())
        cont_list.append(info_cont[i].get_text())
        url_list.append(info_url[i].get('href'))
    return url_list

#解析子网页函数
def child_def(url_list):
    address_list = []
    #进入子网页爬取信息
    for j in range(len(url_list)):  
        child_html = resp_def(url_list[j],ua)
        soup = soup_def(child_html)
        #html = child_html.content
        #soup = BeautifulSoup(html,'html.parser')
        #soup.prettify()
        info_child = soup.select('div.user-info > a')
        if info_child == []:        
            address_list.append('0')
        else:
            address_list.append(info_child[0].get_text())
        #print(info_child)
    print(address_list)
#print(len(title_list))
#print(len(time_list))

#主函数
if  __name__  == '__main__':
    for i in range(4):
        url = 'https://movie.douban.com/subject/26266893/reviews?start=%d'%(i*20)
        ua = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1'}

        url_list = info_def(soup_def(resp_def(url,ua)))
        child_def(url_list)